<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/NCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense

In [ ]:
data=pd.read_html("https://imammarzuki.online/Administrator&password=imammarzuki/")
data[0]

,No,User ID,Daerah Asal,Usia,Jenis Kelamin,Hobi,Status Pernikahan,Pendidikan Terakhir,Pekerjaan,Frekuensi Berwisata Setahun Terakhir,...,BNS,Eco Green Park,Alun-alun Kota Batu,Kusuma Agro,Cangar,Coban Talun,Pemandian Songgoriti,Coban Rais,Predator Fun Park,Tindakan
0,1,1,Luar kota Batu tetapi masih 1 provinsi,36,Laki-Laki,Travelling,Sudah Menikah,S2,Karyawan Swasta,lebih dari 3 kali,...,4,4,3,4,3,3,3,3,4,Edit | Hapus
1,2,2,Luar kota Batu tetapi masih 1 provinsi,33,Perempuan,Hobi Lainnya,Sudah Menikah,S2,PNS,lebih dari 3 kali,...,5,5,4,5,4,5,5,5,5,Edit | Hapus
2,3,3,Luar kota Batu tetapi masih 1 provinsi,45,Laki-Laki,Travelling,Sudah Menikah,S2,PNS,3 kali,...,5,5,4,5,5,5,5,5,5,Edit | Hapus
3,4,4,Luar kota Batu tetapi masih 1 provinsi,41,Perempuan,Travelling,Sudah Menikah,S1/D4,Wiraswasta,3 kali,...,5,5,5,5,5,5,5,5,5,Edit | Hapus
4,5,5,Luar kota Batu tetapi masih 1 provinsi,33,Laki-Laki,Hobi Lainnya,Sudah Menikah,S2,PNS,lebih dari 3 kali,...,3,0,5,3,0,5,5,5,0,Edit | Hapus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,1487,1488,Luar kota Batu tetapi masih 1 provinsi,12,Laki-Laki,Hobi Lainnya,Belum Menikah,S1,Pelajar/Mahasiswa,lebih dari 3 kali,...,5,4,3,4,3,3,3,3,3,Edit | Hapus
1487,1488,1489,Luar kota Batu tetapi masih 1 provinsi,12,Perempuan,Hobi Lainnya,Belum Menikah,S1,Pelajar/Mahasiswa,lebih dari 3 kali,...,5,5,4,5,4,5,5,3,5,Edit | Hapus
1488,1489,1490,Luar kota Batu tetapi masih 1 provinsi,12,Laki-Laki,Travelling,Belum Menikah,S1,Pelajar/Mahasiswa,2 kali,...,4,4,3,4,3,3,3,3,3,Edit | Hapus
1489,1490,1491,Luar kota Batu tetapi masih 1 provinsi,12,Perempuan,Hobi Lainnya,Belum Menikah,S1,Pelajar/Mahasiswa,lebih dari 3 kali,...,5,4,4,5,4,5,5,3,5,Edit | Hapus


In [ ]:
data[0].columns

Index(['No', 'User ID', 'Daerah Asal', 'Usia', 'Jenis Kelamin', 'Hobi',
       'Status Pernikahan', 'Pendidikan Terakhir', 'Pekerjaan',
       'Frekuensi Berwisata Setahun Terakhir',
       'Jenis Wisata yang Sering Dikunjungi', 'Alasan Berwisata',
       'Teman Perjalanan', 'Kendaraan yang Digunakan', 'Menyediakan Anggaran',
       'Jatim Park 1', 'Jatim Park 2', 'Jatim Park 3', 'Museum Angkut',
       'Selecta', 'BNS', 'Eco Green Park', 'Alun-alun Kota Batu',
       'Kusuma Agro', 'Cangar', 'Coban Talun', 'Pemandian Songgoriti',
       'Coban Rais', 'Predator Fun Park', 'Tindakan'],
      dtype='object')

In [ ]:
data1=data[0].rename({"Frekuensi Berwisata Setahun Terakhir":"Frekuensi Berwisata","Jenis Wisata yang Sering Dikunjungi":"Jenis Wisata","Kendaraan yang Digunakan":"Kendaraan"},axis=1)
data1.columns

Index(['No', 'User ID', 'Daerah Asal', 'Usia', 'Jenis Kelamin', 'Hobi',
       'Status Pernikahan', 'Pendidikan Terakhir', 'Pekerjaan',
       'Frekuensi Berwisata', 'Jenis Wisata', 'Alasan Berwisata',
       'Teman Perjalanan', 'Kendaraan', 'Menyediakan Anggaran', 'Jatim Park 1',
       'Jatim Park 2', 'Jatim Park 3', 'Museum Angkut', 'Selecta', 'BNS',
       'Eco Green Park', 'Alun-alun Kota Batu', 'Kusuma Agro', 'Cangar',
       'Coban Talun', 'Pemandian Songgoriti', 'Coban Rais',
       'Predator Fun Park', 'Tindakan'],
      dtype='object')

In [ ]:
df=data1.drop(["Tindakan"], axis=1)
df.columns

Index(['No', 'User ID', 'Daerah Asal', 'Usia', 'Jenis Kelamin', 'Hobi',
       'Status Pernikahan', 'Pendidikan Terakhir', 'Pekerjaan',
       'Frekuensi Berwisata', 'Jenis Wisata', 'Alasan Berwisata',
       'Teman Perjalanan', 'Kendaraan', 'Menyediakan Anggaran', 'Jatim Park 1',
       'Jatim Park 2', 'Jatim Park 3', 'Museum Angkut', 'Selecta', 'BNS',
       'Eco Green Park', 'Alun-alun Kota Batu', 'Kusuma Agro', 'Cangar',
       'Coban Talun', 'Pemandian Songgoriti', 'Coban Rais',
       'Predator Fun Park'],
      dtype='object')

In [ ]:
df

,No,User ID,Daerah Asal,Usia,Jenis Kelamin,Hobi,Status Pernikahan,Pendidikan Terakhir,Pekerjaan,Frekuensi Berwisata,...,Selecta,BNS,Eco Green Park,Alun-alun Kota Batu,Kusuma Agro,Cangar,Coban Talun,Pemandian Songgoriti,Coban Rais,Predator Fun Park
0,1,1,Luar kota Batu tetapi masih 1 provinsi,36,Laki-Laki,Travelling,Sudah Menikah,S2,Karyawan Swasta,lebih dari 3 kali,...,4,4,4,3,4,3,3,3,3,4
1,2,2,Luar kota Batu tetapi masih 1 provinsi,33,Perempuan,Hobi Lainnya,Sudah Menikah,S2,PNS,lebih dari 3 kali,...,5,5,5,4,5,4,5,5,5,5
2,3,3,Luar kota Batu tetapi masih 1 provinsi,45,Laki-Laki,Travelling,Sudah Menikah,S2,PNS,3 kali,...,5,5,5,4,5,5,5,5,5,5
3,4,4,Luar kota Batu tetapi masih 1 provinsi,41,Perempuan,Travelling,Sudah Menikah,S1/D4,Wiraswasta,3 kali,...,5,5,5,5,5,5,5,5,5,5
4,5,5,Luar kota Batu tetapi masih 1 provinsi,33,Laki-Laki,Hobi Lainnya,Sudah Menikah,S2,PNS,lebih dari 3 kali,...,5,3,0,5,3,0,5,5,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,1487,1488,Luar kota Batu tetapi masih 1 provinsi,12,Laki-Laki,Hobi Lainnya,Belum Menikah,S1,Pelajar/Mahasiswa,lebih dari 3 kali,...,4,5,4,3,4,3,3,3,3,3
1487,1488,1489,Luar kota Batu tetapi masih 1 provinsi,12,Perempuan,Hobi Lainnya,Belum Menikah,S1,Pelajar/Mahasiswa,lebih dari 3 kali,...,5,5,5,4,5,4,5,5,3,5
1488,1489,1490,Luar kota Batu tetapi masih 1 provinsi,12,Laki-Laki,Travelling,Belum Menikah,S1,Pelajar/Mahasiswa,2 kali,...,4,4,4,3,4,3,3,3,3,3
1489,1490,1491,Luar kota Batu tetapi masih 1 provinsi,12,Perempuan,Hobi Lainnya,Belum Menikah,S1,Pelajar/Mahasiswa,lebih dari 3 kali,...,5,5,4,4,5,4,5,5,3,5


In [ ]:
# Preprocessing
le = LabelEncoder()
df['Daerah Asal'] = le.fit_transform(df['Daerah Asal'])
df['Jenis Kelamin'] = le.fit_transform(df['Jenis Kelamin'])
df['Hobi'] = le.fit_transform(df['Hobi'])
df['Status Pernikahan'] = le.fit_transform(df['Status Pernikahan'])
df['Pendidikan Terakhir'] = le.fit_transform(df['Pendidikan Terakhir'])
df['Pekerjaan'] = le.fit_transform(df['Pekerjaan'])
df['Frekuensi Berwisata'] = le.fit_transform(df['Frekuensi Berwisata'])
df['Jenis Wisata'] = le.fit_transform(df['Jenis Wisata'])
df['Alasan Berwisata'] = le.fit_transform(df['Alasan Berwisata'])
df['Teman Perjalanan'] = le.fit_transform(df['Teman Perjalanan'])
df['Kendaraan'] = le.fit_transform(df['Kendaraan'])
df['Menyediakan Anggaran'] = le.fit_transform(df['Menyediakan Anggaran'])

scaler = MinMaxScaler()
df[['Usia']] = scaler.fit_transform(df[['Usia']])

In [ ]:
# Preparing the data for NCF
user_features = df[['User ID','Daerah Asal', 'Usia', 'Jenis Kelamin', 'Hobi', 'Status Pernikahan', 'Pendidikan Terakhir', 'Pekerjaan', 'Frekuensi Berwisata', 'Jenis Wisata', 'Alasan Berwisata', 'Teman Perjalanan', 'Menyediakan Anggaran']]
ratings = df[['Jatim Park 1', 'Jatim Park 2', 'Jatim Park 3', 'Museum Angkut', 'Selecta', 'BNS', 'Eco Green Park', 'Alun-alun Kota Batu', 'Kusuma Agro', 'Cangar', 'Coban Talun', 'Pemandian Songgoriti', 'Coban Rais', 'Predator Fun Park']]

In [ ]:
# Reshape data for training
n_users = user_features.shape[0]
n_items = ratings.shape[1]
user_ids = np.array([[i]*n_items for i in range(n_users)]).flatten()
item_ids = np.array([list(range(n_items))]*n_users).flatten()
ratings_flat = ratings.values.flatten()

# Model Architecture
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=n_users, output_dim=50)(user_input)
item_embedding = Embedding(input_dim=n_items, output_dim=50)(item_input)

user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)

input_vecs = Concatenate()([user_vecs, item_vecs])

dense = Dense(128, activation='relu')(input_vecs)
output = Dense(1)(dense)

model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mse')

# Training the model
model.fit([user_ids, item_ids], ratings_flat, epochs=100, verbose=1)

# Predicting ratings
predictions = model.predict([user_ids, item_ids]).flatten()
predictions = predictions.reshape((n_users, n_items))

# Evaluasi
true_ratings = ratings.values
mae = mean_absolute_error(true_ratings, predictions)
rmse = mean_squared_error(true_ratings, predictions, squared=False)

# Untuk metrik classification, kita perlu binarize ratings
threshold = 3.5
true_binary = (true_ratings > threshold).astype(int)
pred_binary = (predictions > threshold).astype(int)

accuracy = accuracy_score(true_binary.flatten(), pred_binary.flatten())
precision = precision_score(true_binary.flatten(), pred_binary.flatten(), average='weighted')
recall = recall_score(true_binary.flatten(), pred_binary.flatten(), average='weighted')
f1 = f1_score(true_binary.flatten(), pred_binary.flatten(), average='weighted')

# Output hasil evaluasi
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


Epoch 1/100
653/653 [==============================] - 4s 4ms/step - loss: 1.4279
Epoch 2/100
653/653 [==============================] - 2s 3ms/step - loss: 0.2545
Epoch 3/100
653/653 [==============================] - 1s 2ms/step - loss: 0.1861
Epoch 4/100
653/653 [==============================] - 1s 2ms/step - loss: 0.1801
Epoch 5/100
653/653 [==============================] - 2s 2ms/step - loss: 0.1765
Epoch 6/100
653/653 [==============================] - 2s 3ms/step - loss: 0.1708
Epoch 7/100
653/653 [==============================] - 2s 3ms/step - loss: 0.1676
Epoch 8/100
653/653 [==============================] - 1s 2ms/step - loss: 0.1636
Epoch 9/100
653/653 [==============================] - 1s 2ms/step - loss: 0.1579
Epoch 10/100
653/653 [==============================] - 1s 2ms/step - loss: 0.1521
Epoch 11/100
653/653 [==============================] - 1s 2ms/step - loss: 0.1430
Epoch 12/100
653/653 [==============================] - 1s 2ms/step - loss: 0.1302
Epoch 13/100
